## Preprocessing

In [45]:
!pip install keras-tuner

In [66]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn as skl
import pandas as pd
import tensorflow as tf
import keras_tuner as kt 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

c:\Users\vaibh\anaconda3\envs\myfirstpythonenv\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  # Remove the CWD from sys.path while we load stuff.


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [47]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
filtered_df = application_df.drop(columns=["NAME", "EIN"])
filtered_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [48]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
filtered_cat = filtered_df.dtypes[filtered_df.dtypes == "object"].index.tolist()
filtered_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [49]:
filtered_df[filtered_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [50]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
#  YOUR CODE GOES HERE
app_type = filtered_df['APPLICATION_TYPE'].value_counts().reset_index()
app_type

,index,APPLICATION_TYPE
0,T3,27037
1,T4,1542
2,T6,1216
3,T5,1173
4,T19,1065
5,T8,737
6,T7,725
7,T10,528
8,T9,156
9,T13,66


In [51]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE
application_types_to_replace = []
application_types_to_replace = app_type[app_type['APPLICATION_TYPE'] <= 156]['index']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [52]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
#  YOUR CODE GOES HERE
# Look at CLASSIFICATION value counts for binning
class_count = filtered_df['CLASSIFICATION'].value_counts().reset_index()
class_count
class_count

classifications_to_replace = []
classifications_to_replace = class_count[class_count['CLASSIFICATION'] < 1883]['index']
# Replace in dataframe
for cls in classifications_to_replace:
    filtered_df['CLASSIFICATION'] = filtered_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
filtered_df['CLASSIFICATION'].value_counts()


C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [53]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#  YOUR CODE GOES HERE
class_count_filtered = []
class_count_filtered = class_count[class_count['CLASSIFICATION'] > 1]
class_count_filtered

,index,CLASSIFICATION
0,C1000,17326
1,C2000,6074
2,C1200,4837
3,C3000,1918
4,C2100,1883
5,C7000,777
6,C1700,287
7,C4000,194
8,C5000,116
9,C1270,114


In [54]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [55]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE
numerical_df = pd.get_dummies(filtered_df)
numerical_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [56]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
y = numerical_df["IS_SUCCESSFUL"].values
X = numerical_df.drop(["IS_SUCCESSFUL"],1).values


# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

c:\Users\vaibh\anaconda3\envs\myfirstpythonenv\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


In [57]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [62]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=120,
        step=5), activation=activation, input_dim=43))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=120,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="relu"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model
     


In [63]:

# Import the kerastuner library
import keras_tuner as kt
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

Reloading Tuner from .\untitled_project\tuner0.json


In [64]:
import math
from functools import reduce
import operator

# Check if math.prod is available
if not hasattr(math, 'prod'):
    def prod(iterable):
        return reduce(operator.mul, iterable, 1)
    math.prod = prod

In [68]:
# Define the HyperModel class
class MyHyperModel(HyperModel):
    def build(self, hp):
        model = Sequential()
        model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32),
                        activation='relu',
                        input_shape=(51,)))  # Set input_shape to match your data
        model.add(Dense(1, activation='sigmoid'))  # Example for binary classification
        model.compile(optimizer='adam',
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
        return model

# Instantiate the HyperModel
hypermodel = MyHyperModel()
# Create the tuner
tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='intro_to_kt'
)

In [69]:
print(X_train_scaled.shape)  # Should print (number_of_samples, 51)
print(X_test_scaled.shape)   # Should print (number_of_samples, 51)

(25724, 51)
(8575, 51)


In [70]:
# Run the kerastuner search for best hyperparameters
#tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))

Trial 5 Complete [00h 00m 55s]
val_accuracy: 0.7316618164380392

Best val_accuracy So Far: 0.732478141784668
Total elapsed time: 00h 04m 23s


In [71]:

# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'units': 320}
{'units': 224}
{'units': 160}


In [72]:

# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5544 - accuracy: 0.7333 - 256ms/epoch - 957us/step
Loss: 0.5543564558029175, Accuracy: 0.7332944869995117
268/268 - 0s - loss: 0.5545 - accuracy: 0.7328 - 261ms/epoch - 975us/step
Loss: 0.55450040102005, Accuracy: 0.7328279614448547
268/268 - 0s - loss: 0.5536 - accuracy: 0.7333 - 255ms/epoch - 952us/step
Loss: 0.5536434054374695, Accuracy: 0.7332944869995117


In [73]:

# Creating model based on the top hyperparameter

nn = tf.keras.models.Sequential()

# Input layer
nn.add(tf.keras.layers.Dense(units = 11, activation = 'relu', input_dim = 43))
# First hidden layer
nn.add(tf.keras.layers.Dense(units = 56, activation = 'relu'))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 71, activation = 'relu'))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units = 11, activation = 'relu'))

nn.add(tf.keras.layers.Dense(units = 96, activation = 'relu'))

nn.add(tf.keras.layers.Dense(units = 31, activation = 'relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])

In [75]:
# Define the model
nn = Sequential()
nn.add(Dense(64, input_shape=(51,), activation='relu'))  # Ensure input_shape matches X_train_scaled
nn.add(Dense(32, activation='relu'))
nn.add(Dense(1, activation='sigmoid'))  # Adjust based on your problem type

# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=20, validation_split=0.2)

Epoch 1/20
644/644 [==============================] - 1s 1ms/step - loss: 0.5801 - accuracy: 0.7149 - val_loss: 0.5510 - val_accuracy: 0.7353
Epoch 2/20
644/644 [==============================] - 1s 1ms/step - loss: 0.5584 - accuracy: 0.7279 - val_loss: 0.5546 - val_accuracy: 0.7359
Epoch 3/20
644/644 [==============================] - 1s 978us/step - loss: 0.5560 - accuracy: 0.7272 - val_loss: 0.5528 - val_accuracy: 0.7320
Epoch 4/20
644/644 [==============================] - 1s 921us/step - loss: 0.5541 - accuracy: 0.7283 - val_loss: 0.5489 - val_accuracy: 0.7361
Epoch 5/20
644/644 [==============================] - 1s 896us/step - loss: 0.5519 - accuracy: 0.7285 - val_loss: 0.5482 - val_accuracy: 0.7355
Epoch 6/20
644/644 [==============================] - 1s 935us/step - loss: 0.5505 - accuracy: 0.7287 - val_loss: 0.5493 - val_accuracy: 0.7337
Epoch 7/20
644/644 [==============================] - 1s 953us/step - loss: 0.5492 - accuracy: 0.7305 - val_loss: 0.5510 - val_accuracy: 0.7

In [ ]:
# Train the model
#fit_model = nn.fit(X_train_scaled,y_train,epochs =20)


In [76]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5560 - accuracy: 0.7306 - 226ms/epoch - 843us/step
Loss: 0.5559728145599365, Accuracy: 0.7306122183799744


In [78]:
# Exporting the model to HDF5 file
from IPython.display import FileLink
nn.save('AlphabetSoupCharity_Optimisation.h5')
display(FileLink('AlphabetSoupCharity_Optimisation.h5'))

c:\Users\vaibh\OneDrive\Desktop\Uni Adelaide Bootcamp\my_homework\deep-learning-challenge\AlphabetSoupCharity_Optimisation.h5